In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
#import pickle
#import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
#import seaborn as sns
#from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
#from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
import glob
#import sys
#np.set_printoptions(threshold=sys.maxsize)

2022-12-09 18:13:00.292072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 18:13:00.292096: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Dados CIC-IDS2017

In [2]:
#Carrega dados do dataset
#monday = pd.read_csv(r"C:\Users\luang\Downloads\CIC-IDS2017 ATUALIZADO\Monday-WorkingHours.csv", skipinitialspace = True)
#tuesday = pd.read_csv(r"C:\Users\luang\Downloads\CIC-IDS2017 ATUALIZADO\Tuesday-WorkingHours.csv", skipinitialspace = True)
#wednesday = pd.read_csv(r"C:\Users\luang\Downloads\CIC-IDS2017 ATUALIZADO\Wednesday-WorkingHours.csv", skipinitialspace = True)
#thursday = pd.read_csv(r"C:\Users\luang\Downloads\CIC-IDS2017 ATUALIZADO\Thursday-WorkingHours.csv", skipinitialspace = True)
#friday = pd.read_csv(r"C:\Users\luang\Downloads\CIC-IDS2017 ATUALIZADO\Friday-WorkingHours.csv", skipinitialspace = True)

# Carregamento Datasets

In [3]:
#Carrega dados do dataset
monday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Monday-WorkingHours.csv", skipinitialspace = True)
tuesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Tuesday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((monday, tuesday), axis=0)
del monday, tuesday

In [4]:
wednesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Wednesday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, wednesday), axis=0)
del wednesday

In [5]:
thursday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Thursday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, thursday), axis=0)
del thursday

In [6]:
friday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Friday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, friday), axis=0)
del friday

In [7]:
#Concatena todos os datasets carregados

#x = pd.concat((monday, friday1, friday2, friday3), axis=0)
#x = pd.concat((monday, tuesday, wednesday, thursday, friday), axis=0)
x = x.drop(['Flow ID', 'Src IP', 'Dst IP', 'Src Port', 'Dst Port', 'Timestamp'], axis = 1)

In [8]:
#Mostra todas as classes possíveis para a label
#x['Label'].unique()

In [9]:
#Transforma o dataset multiclasse para binário
x['Label'] = x['Label'].replace('BENIGN', 0)
x['Label'] = x['Label'].replace('FTP-Patator', 1)
x['Label'] = x['Label'].replace('SSH-Patator', 1)
x['Label'] = x['Label'].replace('FTP-Patator - Attempted', 0)
x['Label'] = x['Label'].replace('SSH-Patator - Attempted', 0)
x['Label'] = x['Label'].replace('DoS slowloris', 1)
x['Label'] = x['Label'].replace('DoS slowloris - Attempted', 0)
x['Label'] = x['Label'].replace('DoS Slowhttptest', 1)
x['Label'] = x['Label'].replace('DoS Slowhttptest - Attempted', 0)
x['Label'] = x['Label'].replace('DoS Hulk', 1)
x['Label'] = x['Label'].replace('DoS Hulk - Attempted', 0)
x['Label'] = x['Label'].replace('DoS GoldenEye', 1)
x['Label'] = x['Label'].replace('Heartbleed', 1)
x['Label'] = x['Label'].replace('DoS GoldenEye - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - Brute Force', 1)
x['Label'] = x['Label'].replace('Web Attack - Brute Force - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - XSS', 1)
x['Label'] = x['Label'].replace('Web Attack - XSS - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - Sql Injection', 1)
x['Label'] = x['Label'].replace('Infiltration', 1)
x['Label'] = x['Label'].replace('Infiltration - Attempted', 0)
x['Label'] = x['Label'].replace('Bot', 1)
x['Label'] = x['Label'].replace('Bot - Attempted', 0)
x['Label'] = x['Label'].replace('DDoS', 1)
x['Label'] = x['Label'].replace('PortScan', 1)

In [10]:
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.dropna(inplace=True)

In [11]:
#Realiza o One-Hot Encoding
#x = pd.get_dummies(x)

In [12]:
#Separa as labels e os Dados em datasets diferentes
data_class = x['Label']
data = x.drop(['Label'], axis = 1) 

In [13]:
data = data.to_numpy()
#transforma dados infinitos em finitos
#data = np.nan_to_num(data,posinf=10000000, neginf=10000000)
data_class = data_class.to_numpy()

In [14]:
noTrain = np.count_nonzero(data_class == 0)
anTrain = np.count_nonzero(data_class == 1)

In [15]:
print("Dados Normais: ", noTrain, "\nDados Anômalos: ", anTrain)

Dados Normais:  1666172 
Dados Anômalos:  433849


# Abordagem Autoencoder-Algoritmos

In [16]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)
Z = np.arange(0.000,4.095,0.005)
bestZ = np.zeros(num_folds)
bestTeta = np.zeros(num_folds)
bestmean = np.zeros(num_folds)
bestdesv = np.zeros(num_folds)
bestErrorNumber = np.zeros(num_folds)
ErrorNumberFolds = np.zeros((num_folds,1,Z.shape[0]-1))

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
        
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(anTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, noTrain)) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    autoencoderTrain_class = np.zeros(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(int(var/2), activation=activation)(input_vector)
    en2 = layers.Dense(int(var/4), activation=activation)(en1)
    de1 = layers.Dense(int(var/2), activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    print('0')
    #####################################################
    ####Erro de Reconstrução
    #Teste
    pred_ErroRecons = autoencoder.predict(modelTest)
    
    #Extração do erro
    mse_ErroRecons = np.mean(np.power(modelTest - pred_ErroRecons, 2), axis=1)
    error_df_ErroRecons = pd.DataFrame({'reconstruction_error': mse_ErroRecons,
                                        'true_class': modelTest_class})

    media = np.mean(error_df_ErroRecons['reconstruction_error'])
    desv = np.std(error_df_ErroRecons['reconstruction_error'])

    #Determinação do limiar ótimo

    ErrosLimiar = np.zeros(Z.shape[0]-1)
    melhorTeta = 0
    melhorZ = 0
    prediMatrix = np.zeros((Z.shape[0]-1, 1, modelTest.shape[0]))
    
    for i in range(0, Z.shape[0]-1):
        #Obtenção do Limiar
        teta = media + Z[i]*desv
    
        #Predição
        predi_ErroRecons = np.ones(modelTest.shape[0])
        error_ErroRecons = error_df_ErroRecons.to_numpy()
        predi_ErroRecons[np.where(error_ErroRecons[:,0] <= teta)] = 0
        prediMatrix[i] = predi_ErroRecons 
        
        #Contagem da quantidade de erros obtidos para cada Z
        numberErros = 0
        for j in range(0, predi_ErroRecons.shape[0]):
            if error_ErroRecons[j,1] != predi_ErroRecons[j]:
                numberErros = numberErros + 1
        ErrosLimiar[i] = numberErros
        
    predi_ErroRecons = prediMatrix[np.argpartition(ErrosLimiar,1)[0]]
            
    #Métricas
    accuracy_ErroRecons[count] = accuracy_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    recall_ErroRecons[count] = recall_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    precision_ErroRecons[count] = precision_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    f1_ErroRecons[count] = f1_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    matrix_ErroRecons[count] = confusion_matrix(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    bestZ[count] = Z[np.argpartition(ErrosLimiar,1)[0]]
    bestTeta[count] = media + Z[np.argpartition(ErrosLimiar,1)[0]]*desv
    bestmean[count] = media
    bestdesv[count] = desv
    bestErrorNumber[count] = ErrosLimiar[np.argpartition(ErrosLimiar,1)[0]]
    ErrorNumberFolds[count] = ErrosLimiar
    
    print('1')
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class})
    error_knn = error_df_knn.to_numpy()
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_knn[:,0].reshape(-1,1), error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class})
    error_knn = error_df_knn.to_numpy()
    predict_knn = neigh.predict(error_knn[:,0].reshape(-1,1))
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
    
   
    print('2')
    #####################################################
    ####Decision Tree
    ###Treinamento Decision Tree 
    pred_dt = autoencoder.predict(knnTrain)
    mse_dt = np.mean(np.power(knnTrain - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': knnTrain_class})
    error_dt = error_df_dt.to_numpy()
    dt = tree.DecisionTreeClassifier()
    dt.fit(error_dt[:,0].reshape(-1,1), error_dt[:,1])

    #Teste
    pred_dt = autoencoder.predict(modelTest)
    mse_dt = np.mean(np.power(modelTest - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': modelTest_class})
    error_dt = error_df_dt.to_numpy()
    predict_dt = dt.predict(error_dt[:,0].reshape(-1,1))

    #Métricas
    accuracy_dt[count] = accuracy_score(error_dt[:,1], predict_dt[:])
    recall_dt[count] = recall_score(error_dt[:,1], predict_dt[:])
    precision_dt[count] = precision_score(error_dt[:,1], predict_dt[:])
    f1_dt[count] = f1_score(error_dt[:,1], predict_dt[:])
    matrix_dt[count] = confusion_matrix(error_dt[:,1], predict_dt[:])
    
   
    print('3')
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': knnTrain_class})
    error_svm = error_df_svm.to_numpy()
    clf = svm.SVC(kernel='rbf')
    print('4')
    clf.fit(error_svm[:,0].reshape(-1,1), error_svm[:,1])
    print('5')
    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': modelTest_class})
    error_svm = error_df_svm.to_numpy()
    print('6')
    predict_svm = clf.predict(error_svm[:,0].reshape(-1,1))
    print('7')
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])
      
    Result_GridSearchLimiar = np.array(["Referência", "Acurácia", accuracy_ErroRecons[count], "",
                                        "Recall", recall_ErroRecons[count], "",
                                        "Precision", precision_ErroRecons[count], "",
                                        "F1", f1_ErroRecons[count], 
                                        "BestZ", bestZ[count],
                                        "BestTetha", bestTeta[count], 
                                        "BestMean", bestmean[count], 
                                        "BestDesv", bestdesv[count],
                                        "BestErrorNumber", bestErrorNumber[count], "-----------------","-----------------",
                                    "KNN", "Acurácia", accuracy_knn[count], "",
                                        "Recall", recall_knn[count], "",
                                        "Precision", precision_knn[count], "",
                                        "F1", f1_knn[count], "-----------------","-----------------",
                                    "Decision Tree",  "Acurácia", accuracy_dt[count],"",
                                        "Recall", recall_dt[count], "",
                                        "Precision", precision_dt[count], "",
                                        "F1", f1_dt[count], "-----------------","-----------------",
                                    "SVM",  "Acurácia", accuracy_svm[count],"",
                                        "Recall", recall_svm[count], "",
                                        "Precision", precision_svm[count], "",
                                        "F1", f1_svm[count], "-----------------","-----------------","-----------------","-----------------",
                                    "KNN", matrix_knn[count], "-----------------","-----------------",
                                    "Decision Tree", matrix_dt[count], "-----------------","-----------------",
                                    "SVM", matrix_svm[count]])

    np.savetxt('Result_Parcial_CICIDS2017_RE_Fold'+str(fold_no)+'.txt', Result_GridSearchLimiar, delimiter = ',',fmt='%s')
    
    count = count + 1
    fold_no = fold_no + 1

2022-12-09 18:13:58.472958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 18:13:58.473081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-09 18:13:58.473197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-09 18:13:58.473286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-12-09 18:13:58.473374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Epoch 1/50
4333/4333 [==============================] - 12s 3ms/step - loss: 0.0035
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0019
Epoch 3/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0019
Epoch 4/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0019
Epoch 5/50
4333/4333 [==============================] - 11s 3ms/step - loss: 0.0019
Epoch 6/50
4333/4333 [==============================] - 11s 3ms/step - loss: 0.0019
Epoch 7/50
4333/4333 [==============================] - 11s 3ms/step - loss: 0.0019
Epoch 8/50
4333/4333 [==============================] - 12s 3ms/step - loss: 0.0019
Epoch 9/50
4333/4333 [==============================] - 11s 3ms/step - loss: 0.0019
Epoch 10/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0019
Epoch 11/50
4333/4333 [==============================] - 11s 3ms/step - loss: 0.0019
Epoch 12/50
4333/4333 [==============================] - 13s 3ms/step - lo

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:275: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0014
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 7.0534e-04
Epoch 3/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.8728e-04
Epoch 4/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.8024e-04
Epoch 5/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.7675e-04
Epoch 6/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.7489e-04
Epoch 7/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.7340e-04
Epoch 8/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.7222e-04
Epoch 9/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.7075e-04
Epoch 10/50
4333/4333 [==============================] - 11s 2ms/step - loss: 6.7002e-04
Epoch 11/50
4333/4333 [==============================] - 10s 2ms/step - loss: 6.6965e-04
Epoch 12/50
4333/4333 [===========

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:275: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


...

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:275: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0035
Epoch 2/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0031
Epoch 3/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 4/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 5/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 6/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0030
Epoch 7/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 8/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 9/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 10/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 11/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0030
Epoch 12/50
4333/4333 [==============================] - 11s 2ms/step - lo

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:275: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0023
Epoch 2/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0019
Epoch 3/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 4/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 5/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 6/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 7/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 8/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 9/50
4333/4333 [==============================] - 10s 2ms/step - loss: 0.0018
Epoch 10/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 11/50
4333/4333 [==============================] - 11s 2ms/step - loss: 0.0018
Epoch 12/50
4333/4333 [==============================] - 11s 2ms/step - lo

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:275: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [17]:
print("Autoencoder - Limiar Referência:\n",np.mean(matrix_ErroRecons, axis=0), "\nAcurácia: ", np.mean(accuracy_ErroRecons),' +/- ',np.std(accuracy_ErroRecons), "\nRecall: ", np.mean(recall_ErroRecons),' +/- ',np.std(recall_ErroRecons), "\nPrecision: ", np.mean(precision_ErroRecons),' +/- ',np.std(precision_ErroRecons), "\nF1: ", np.mean(f1_ErroRecons),' +/- ',np.std(f1_ErroRecons))
for i in range(0,fold_no-1):
    print("\nFold ",i+1,": " , bestTeta[i], "=", bestmean[i], "+", bestZ[i], "*", bestdesv[i], "\n\nNúmero de Erros: ", bestErrorNumber[i])

Autoencoder - Limiar Referência:
 [[158863.4   7753.8]
 [ 33344.4  10040.5]] 
Acurácia:  0.8042962332296902  +/-  0.0239864273948601 
Recall:  0.2314292417862352  +/-  0.2576757593196485 
Precision:  0.3427432423624391  +/-  0.2516807470899969 
F1:  0.25247359810049724  +/-  0.2646266003061834

Fold  1 :  0.003641944987704381 = 0.0024855516786404085 + 0.23500000000000001 * 0.004920822591761584 

Número de Erros:  36899.0

Fold  2 :  0.001957107793902135 = 0.000998637333409565 + 0.405 * 0.002366593729611284 

Número de Erros:  31738.0

Fold  3 :  0.009009764798907183 = 0.00047946763378260265 + 4.08 * 0.002090759109099162 

Número de Erros:  45486.0

Fold  4 :  0.007098892013852253 = 0.0055669253657172225 + 0.33 * 0.004642323176166761 

Número de Erros:  39617.0

Fold  5 :  0.017364060357410234 = 0.0015475400923381903 + 4.085 * 0.0038718531860641478 

Número de Erros:  44860.0

Fold  6 :  0.010315161331655198 = 0.000388028157884453 + 4.085 * 0.002430142759797 

Número de Erros:  45796.0


In [18]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[151443.6  15173.6]
 [  7206.8  36178.1]] 
Acurácia:  0.8934277159759108  +/-  0.04609604474525712 
Recall:  0.8338869704161933  +/-  0.04645973314528371 
Precision:  0.7192280982700283  +/-  0.11308886801374009 
F1:  0.7692788868469953  +/-  0.08281200722990642

Autoencoder - SVM:
 [[140482.7  26134.5]
 [ 14434.3  28950.6]] 
Acurácia:  0.806817159953314  +/-  0.05360538222325711 
Recall:  0.6672973925760781  +/-  0.1734625885755789 
Precision:  0.5377264048286412  +/-  0.1147346948943886 
F1:  0.5850707211165451  +/-  0.1126219559638405

Autoencoder - Decision Tree:
 [[154674.3  11942.9]
 [ 19885.8  23499.1]] 
Acurácia:  0.8484362487040418  +/-  0.058421288657891146 
Recall:  0.5416424772742733  +/-  0.26967897210943065 
Precision:  0.6664307618076611  +/-  0.12010715491192822 
F1:  0.5612196036028575  +/-  0.2175222991869618


In [19]:
Result_GridSearchLimiar = np.array(["Theta: " , bestTeta,"",
                                "Média: ", bestmean,"",
                                "Valor Z: ", bestZ,"",
                                "Desvio Padrao: ", bestdesv,"",
                                "Número de Erros: ", bestErrorNumber,"",
                                "Erros por Fold: ", ErrorNumberFolds])

np.savetxt('Result_Error_GridSearchLimiarRef.txt', Result_GridSearchLimiar, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [20]:
Result = np.array(["Referência", "Acurácia", accuracy_ErroRecons, np.mean(accuracy_ErroRecons), np.std(accuracy_ErroRecons),"",
                                "Recall", recall_ErroRecons, np.mean(recall_ErroRecons), np.std(recall_ErroRecons), "",
                                "Precision", precision_ErroRecons, np.mean(precision_ErroRecons), np.std(precision_ErroRecons), "",
                                "F1", f1_ErroRecons, np.mean(f1_ErroRecons), np.std(f1_ErroRecons), "-----------------","-----------------",
                   "KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_Error.txt', Result, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [21]:
Result_Matrix = np.array(["Referência", matrix_ErroRecons, np.mean(matrix_ErroRecons, axis=0), np.std(matrix_ErroRecons, axis=0), "-----------------","-----------------",
                          "KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_Error_Matrix.txt', Result_Matrix, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
